In [1]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns 

In [4]:
Case_game = pd.read_csv("Game 9 Case.xlsx - Case.csv")
Case_game = Case_game.loc[:, Case_game.columns != 'Player Name']

In [5]:
Case_full = Case_game[Case_game['Split Name']== 'game']
Case_full

,Session Title,Player Code,Minutes,Split Name,Player Code.1,Distance (miles),Player Code.2,Sprint Distance (yards),Player Code.3,Power Plays,...,Accelerations Zone Count: 3 - 4 m/s/s,Accelerations Zone Count: > 4 m/s/s,Deceleration Zone Count: 0 - 1 m/s/s,Deceleration Zone Count: 1 - 2 m/s/s,Deceleration Zone Count: 2 - 3 m/s/s,Deceleration Zone Count: 3 - 4 m/s/s,Deceleration Zone Count: > 4 m/s/s,Unnamed: 101,Unnamed: 102,Player Code.5
0,Case Western,F,54.0,game,F,2.0810,F,0.0000,F,0,...,5,1,0,69,23,11,4,15.0,15.0,F
1,Case Western,L,36.0,game,L,3.1367,L,290.2233,L,38,...,29,4,0,149,77,46,14,60.0,60.0,L
2,Case Western,X,36.0,game,X,1.4056,X,1.6842,X,1,...,17,3,0,42,26,9,4,13.0,13.0,X
3,Case Western,B,34.0,game,B,2.8015,B,252.7256,B,21,...,17,3,0,109,65,27,18,45.0,45.0,B
4,Case Western,C,28.0,game,C,2.3371,C,74.2572,C,27,...,14,4,0,112,74,19,4,23.0,23.0,C
5,Case Western,K,90.0,game,K,6.0791,K,296.0479,K,56,...,49,6,0,279,141,55,18,73.0,73.0,K
6,Case Western,A,0.0,game,A,0.1389,A,0.0000,A,0,...,0,0,0,0,0,0,0,0.0,0.0,A
7,Case Western,M,90.0,game,M,5.9889,M,688.4166,M,54,...,38,9,0,241,144,35,33,68.0,68.0,M
8,Case Western,U,56.0,game,U,4.2111,U,344.3264,U,44,...,30,5,0,159,88,60,30,90.0,90.0,U
9,Case Western,R,90.0,game,R,6.9335,R,852.7982,R,69,...,31,6,0,273,166,63,22,85.0,85.0,R


In [10]:
Case_volume_intensity = Case_full[["Player Code", "Minutes", "Distance (miles)", "Sprint Distance (yards)", "Power Plays", "Energy (kcal)", "Impacts", "Top Speed (mph)","Distance Per Min (yd/min)", "Power Score (w/kg)", "Player Load", "Work Ratio"]]

In [ ]:
Case_cluster = Case_volume_intensity.loc[:, Case_volume_intensity.columns != 'Player Code']

In [ ]:
Case_distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(Case_cluster)
    Case_distortions.append(kmeanModel.inertia_)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(K, Case_distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method shwoing the optimal k')
plt.show()

In [ ]:
x_Case = Case_cluster.values # numpy array
scaler = preprocessing.MinMaxScaler()
x_scaled = scaler.fit_transform(x_Case)
X_norm = pd.DataFrame(x_scaled)

In [ ]:
pca_Case = PCA(n_components = 2) # 2D PCA for the plot
reduced_Case= pd.DataFrame(pca_Case.fit_transform(X_norm))

In [ ]:
kmeans = KMeans(n_clusters=3)
# fit the input data
kmeans = kmeans.fit(reduced_Case)
# get the cluster labels
labels = kmeans.predict(reduced_Case)
# centroid values
centroid = kmeans.cluster_centers_
# cluster values
clusters = kmeans.labels_.tolist()
# adding names
Case_names = Case_full['Player Code']

In [ ]:
reduced_Case['cluster'] = clusters
reduced_Case['names'] = Case_names
reduced_Case.columns = ['x', 'y', 'cluster', 'names']
reduced_Case.head()

In [ ]:
%matplotlib inline
sns.set(style="white")
ax = sns.lmplot(x="x", y="y", hue='cluster', data = reduced_Case, legend= True,
fit_reg=False, height = 15, scatter_kws={"s": 250})
texts = []
for x, y, s in zip(reduced_Case.x, reduced_Case.y, reduced_Case.names):
    texts.append(plt.text(x, y, s))
ax.set(ylim=(-2, 2))
plt.tick_params(labelsize=15)
plt.xlabel(" Case PC 1", fontsize = 20)
plt.ylabel(" Case PC 2", fontsize = 20)
plt.show()

In [ ]:
Case_group_0 = reduced_Case[(reduced_Case['cluster'] == 0)]
Case_group_0

In [ ]:
Case_group_1 = reduced_Case[(reduced_Case['cluster'] == 1)]
Case_group_1

In [ ]:
Case_group_2 = reduced_Case[(reduced_Case['cluster'] == 2)]
Case_group_2